# Run a suite of models and apply them to test data sets and whole tiles

## Setup

In [1]:
%load_ext autoreload
%autoreload 2

import os
import glob
import run_models

train = run_models.TrainingData()

Using TensorFlow backend.


In [2]:
"""
Paths to the feature (LiDAR), response (manually labelled buildings) and boundary files, and to
the directory where the output will be written
"""

base_path = '/data/gdcsdata/HawaiiMapping/ProjectFiles/Rachel/'
boundary_path = base_path+'labeled_region_boundaries/'
feature_path = base_path+'labeled_region_features/'
response_path = base_path+'labeled_region_buildings/'

paths = {'boundary': boundary_path, 'responses': response_path, 'features': feature_path}

bfgn_output_path = base_path+'bfgn_output_buildings/'



In [3]:
"""
Count the number of pixels and buildings in the training data set
"""

training_sets = {'HBMain': 'HBMain',\
                           'HBLower': 'HBLower',\
                           'HOVE1': 'tile031_3125_11250',\
                           'CC1': 'tile024_10000_3125',\
                           'MIL1': 'tile030_10000_5625',\
                           'Hamakua': 'tile016_0_4375',\
                           'KParadise': 'KParadise',\
                           'CCTrees': 'tile024_10000_4375',\
                           'WAI1': 'Waikoloa1',\
                           'KK1': 'Kukio1',\
                           'Waimea': 'Waimea',\
                           'KonaMauka': 'KonaMauka'}

train.count_buildings(feature_path, response_path, training_sets)

HBLower contains 7650467 pixels and 920 features
HOVE1 contains 1568750 pixels and 151 features
CC1 contains 1562500 pixels and 160 features
MIL1 contains 1562500 pixels and 102 features
Hamakua contains 1570008 pixels and 64 features
KParadise contains 6827625 pixels and 339 features
CCTrees contains 1571260 pixels and 1 features
WAI1 contains 1177692 pixels and 167 features
KK1 contains 1181043 pixels and 80 features
Waimea contains 487602 pixels and 168 features
KonaMauka contains 7818572 pixels and 1 features
Total number of features = 2153
Total number of pixels = 32978019


In [4]:
test_sets = {'HBTest': 'HBTest',\
                       'HOVE2': 'tile031_2500_11250',\
                       'MIL2': 'tile030_9375_5625',\
                       'CC2': 'tile024_10000_2500',\
                       'SKona_A': 'SKona_TestA',\
                       'SKona_B': 'SKona_TestB',\
                       'Hamakua_A': 'Hamakua_testA',\
                       'Puako': 'Puako'}

train.count_buildings(feature_path, response_path, test_sets)

HBTest contains 4709250 pixels and 416 features
HOVE2 contains 1562500 pixels and 194 features
MIL2 contains 765044 pixels and 52 features
CC2 contains 1562500 pixels and 270 features
SKona_A contains 603411 pixels and 59 features
SKona_B contains 605453 pixels and 59 features
Hamakua_A contains 4814024 pixels and 130 features
Puako contains 1838160 pixels and 144 features
Total number of features = 1324
Total number of pixels = 16460342


## Run the model suite

The test datasets and model parameters vary like this:

Data types:
- 1m DSM
- 1m eigenvalues, all 4 bands
- 1m hillshade

Training data:
- all available labelled training regions

Window sizes:
- 16, 32, 64 pixels

In [5]:

data_types = ['hires_surface', 'eigen4', 'hillshade']
training_set = []
for _ in range(len(data_types)):
    training_set.append(['HBLower', 'HOVE1', 'MIL1', 'CC1', 'Hamakua', 'KParadise', 'CCTrees', 'WAI1', 'KK1',\
                        'Waimea', 'KonaMauka'])

training_data = train.create_training_lists(paths, training_sets, training_set)

permutations = dict.fromkeys(['boundary_files', 'feature_files', 'response_files', 'window_radius',\
                              'loss_window_radius']) 

#This gives 1 set of training regions, 3 window sizes (w/ appropriate loss_window_radius)
parameter_combos = [(training_data[0][0], training_data[0][1], training_data[0][2], 16, 8),\
                    (training_data[0][0], training_data[0][1], training_data[0][2], 32, 16),\
                    (training_data[0][0], training_data[0][1], training_data[0][2], 64, 32)]

#This adds the different data types - DSM, eigenvals, hillshade
for kind in data_types[1:]:
    for data in parameter_combos[:3]:
        new = [[x[0].replace('hires_surface', kind)] for x in data[1]]
    parameter_combos.extend([(training_data[0][0], new, training_data[0][2], 16, 8),\
                    (training_data[0][0], new, training_data[0][2], 32, 16),\
                    (training_data[0][0], new, training_data[0][2], 64, 32)])

print(f"There are {len(parameter_combos)} parameter combinations")
            
iteration_data = {} 
iteration_data['permutations'] = permutations
iteration_data['parameter_combos'] = parameter_combos
iteration_data['nicknames'] = ['hires_surface_16', 'eigen4_16', 'hillshade_16', 'hires_surface_32', 'eigen4_32',\
                               'hillshade_32', 'hires_surface_64', 'eigen4_64', 'hillshade_64']
iteration_data['out_path'] = bfgn_output_path+'model_runs/'
iteration_data['data_types'] = data_types

model_runs = run_models.Loops(iteration_data, settings_file='settings_buildings.yaml')

There are 9 parameter combinations


In [6]:
%%script echo skipping
%%time
model_runs.loop_over_configs(use_existing=True, rebuild_data=True, fit_model=True)

skipping


## Apply the models to the test regions and the whole (trimmed) tiles

In [ ]:
"""
The test regions
"""

model_paths = sorted(glob.glob(model_runs.out_path+'combo_*'))
for model_dir, combo in zip(model_paths, parameter_combos):
    config_file = f'{model_dir}/config.yaml'
    for idx, test_data in enumerate(test_sets.keys()):
        for kind in data_types:
            if kind in combo[1][idx][0].split('/')[-1]:
                replace_with = kind
        application_data = f'{feature_path}{test_sets[test_data]}_{replace_with}.tif'
        outfile = f'{model_dir}/applied_model_{test_data}'
        model_runs.apply_model(config_file, application_data, outfile)
    

Working on /data/gdcsdata/HawaiiMapping/ProjectFiles/Rachel/labeled_region_features/HBTest_hires_surface.tif, writing /data/gdcsdata/HawaiiMapping/ProjectFiles/Rachel/bfgn_output_buildings/model_runs/combo_0/applied_model_HBTest


100%|█████████████████████████████████████████| 122/122 [00:11<00:00, 10.94it/s]


/data/gdcsdata/HawaiiMapping/ProjectFiles/Rachel/bfgn_output_buildings/model_runs/combo_0/applied_model_HBTest.tif
Working on /data/gdcsdata/HawaiiMapping/ProjectFiles/Rachel/labeled_region_features/tile031_2500_11250_hires_surface.tif, writing /data/gdcsdata/HawaiiMapping/ProjectFiles/Rachel/bfgn_output_buildings/model_runs/combo_0/applied_model_HOVE2


100%|███████████████████████████████████████████| 79/79 [00:03<00:00, 20.17it/s]


/data/gdcsdata/HawaiiMapping/ProjectFiles/Rachel/bfgn_output_buildings/model_runs/combo_0/applied_model_HOVE2.tif
Working on /data/gdcsdata/HawaiiMapping/ProjectFiles/Rachel/labeled_region_features/tile030_9375_5625_hires_surface.tif, writing /data/gdcsdata/HawaiiMapping/ProjectFiles/Rachel/bfgn_output_buildings/model_runs/combo_0/applied_model_MIL2


...

/data/gdcsdata/HawaiiMapping/ProjectFiles/Rachel/bfgn_output_buildings/model_runs/combo_2/applied_model_MIL2.tif
Working on /data/gdcsdata/HawaiiMapping/ProjectFiles/Rachel/labeled_region_features/tile024_10000_2500_hires_surface.tif, writing /data/gdcsdata/HawaiiMapping/ProjectFiles/Rachel/bfgn_output_buildings/model_runs/combo_2/applied_model_CC2


100%|███████████████████████████████████████████| 20/20 [00:08<00:00,  2.44it/s]


/data/gdcsdata/HawaiiMapping/ProjectFiles/Rachel/bfgn_output_buildings/model_runs/combo_2/applied_model_CC2.tif
Working on /data/gdcsdata/HawaiiMapping/ProjectFiles/Rachel/labeled_region_features/SKona_TestA_hires_surface.tif, writing /data/gdcsdata/HawaiiMapping/ProjectFiles/Rachel/bfgn_output_buildings/model_runs/combo_2/applied_model_SKona_A


100%|███████████████████████████████████████████| 10/10 [00:07<00:00,  1.39it/s]


/data/gdcsdata/HawaiiMapping/ProjectFiles/Rachel/bfgn_output_buildings/model_runs/combo_2/applied_model_SKona_A.tif


In [ ]:
"""
The entire tiles (mosaics??)
"""